In [1]:
!pip install pandas
!pip install requests
!pip install bs4
!pip install plotly
!pip install numpy
!pip install geopy
!pip install html5lib
!pip install sqlalchemy==1.3.9
!pip install ipython-sql
!pip install psycopg2-binary

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import clear_output
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import html5lib
import psycopg2
import json

In [3]:
# load data from json

# with open('Data_to_Otomoto.json') as file:
#     Data_to_Otomoto = json.loads(file.read())

# car_brand = Data_to_Otomoto["car_brand"]

In [4]:
insert_table = ['ID',
    'Oferta_od',
    'Kategoria',
    'Marka_pojazdu',
    'Model_pojazdu',
    'Wersja',
    'Generacja',
    'Rok_produkcji',
    'Przebieg',
    'Pojemnosc_skokowa',
    'Rodzaj_paliwa',
    'Moc',
    'Skrzynia_biegow',
    'Naped',
    'Typ_nadwozia',
    'Emisja_CO2',
    'Liczba_drzwi',
    'Liczba_miejsc',
    'Kolor',
    'Rodzaj_koloru',
    'VAT_marza',
    'Zarejestrowany_w_Polsce',
    'Pierwszy_wlasciciel',
    'Bezwypadkowy',
    'Stan',
    'Kraj_pochodzenia',
    'Pierwsza_rejestracja',
    'Serwisowany_w_ASO',
    'Mozliwosc_finansowania',
    'Faktura_VAT',
    'Okres_gwarancji_producenta',
    'Spalanie_W_Miescie',
    'Numer_rejestracyjny_pojazdu',
    'Leasing',
    'Filtr_czastek_stalych',
    'Spalanie_W_Cyklu_Mieszanym',
    'Spalanie_Poza_Miastem',
    'Uszkodzony',
    'Gwarancja_dealerska_w_cenie',
    'Miesieczna_rata']

In [5]:
html_data = requests.get("https://www.otomoto.pl/oferta/dacia-duster-prestige-130km-faktura-vat-fabryczna-gwarancja-salon-pl-u-ID6DQcyU.html#7ce9f71ee3")
soup = BeautifulSoup(html_data.content, 'html5lib')

In [6]:
eur = 4.57
print(eur)


4.57


In [7]:
def get_price(soup):
    price = soup.find('span',attrs={"class":"offer-price__number"}).text
    price = price.replace(" ","").replace(",",".")
    if "PLN" in price:
        price = price.replace("PLN","")
        price = float(price)
    elif "EUR" in price:
        price = price.replace("EUR","")
        price = float(price)
        price = price * eur
    else:
        price = None
    return price

In [8]:
print(get_price(soup))

69900.0


In [9]:
def take_id(soup):
    ID = soup.find('div',attrs={"class":"offer-content__metabar"})
    ID = ID.find('div')
    ID = ID.find('span',attrs={"id":"ad_id"}).text
    return ID

In [10]:
def take_details(soup):
    details = {}    
    for info in soup.find_all('li',attrs={"class":"offer-params__item"}):
        key = info.find('span').text.strip().replace(" ","_").replace("(","").replace(")","")
        key = key.replace("ś","s").replace("ż","z").replace("ó","o").replace("ć","c").replace("ę","e")
        key = key.replace("ł","l")
        if key in insert_table:
            value = info.find('div').text.strip()
            details[key] = value
    return details

In [11]:
def take_all(soup):
    try:
        dict_all = {}
        dict_all["ID"] = take_id(soup)
        dict_all["price"] = get_price(soup)
        details = take_details(soup)
        return dict_all | details
    except:
        pass

In [12]:
def data_clean(details):
    try:
        details['Przebieg'] = details['Przebieg'].replace("km","").replace(" ","")
    except:
        pass
    try:
        details['Moc'] = details['Moc'].replace("KM","").replace(" ","").replace(",",".")
    except:
        pass
    try:
        details['Pojemnosc_skokowa'] = details['Pojemnosc_skokowa'].replace("cm3","").replace(" ","").replace(",",".")
    except:
        pass
    try:
        details['Spalanie_W_Miescie'] = details['Spalanie_W_Miescie'].replace("l/100km","").replace(" ","").replace(",",".")
    except:
        pass
    try:
        details['Spalanie_W_Cyklu_Mieszanym'] = details['Spalanie_W_Cyklu_Mieszanym'].replace("l/100km","").replace(" ","").replace(",",".")
    except:
        pass
    try:
        details['Spalanie_Poza_Miastem'] = details['Spalanie_Poza_Miastem'].replace("l/100km","").replace(" ","").replace(",",".")
    except:
        pass
    return details

In [13]:
car_brand = [
  "Abarth",
  "Alfa Romeo",
  "Aston Martin",
  "Audi",
  "Bentley",
  "BMW"
#   "Bugatti",
#   "Cadillac",
#   "Chevrolet",
#   "Chrysler",
#   "Citroën",
#   "Dacia",
#   "Daewoo",
#   "Daihatsu",
#   "Dodge",
#   "Donkervoort",
#   "DS",
#   "Ferrari",
#   "Fiat",
#   "Fisker",
#   "Ford",
#   "Honda",
#   "Hummer",
#   "Hyundai",
#   "Infiniti",
#   "Iveco",
#   "Jaguar",
#   "Jeep",
#   "Kia",
#   "KTM",
#   "Lada",
#   "Lamborghini",
#   "Lancia",
#   "Land Rover",
#   "Landwind",
#   "Lexus",
#   "Lotus",
#   "Maserati",
#   "Maybach",
#   "Mazda",
#   "McLaren",
#   "Mercedes-Benz",
#   "MG",
#   "Mini",
#   "Mitsubishi",
#   "Morgan",
#   "Nissan",
#   "Opel",
#   "Peugeot",
#   "Porsche",
#   "Renault",
#   "Rolls-Royce",
#   "Rover",
#   "Saab",
#   "Seat",
#   "Skoda",
#   "Smart",
#   "SsangYong",
#   "Subaru",
#   "Suzuki",
#   "Tesla",
#   "Toyota",
#   "Volkswagen",
#   "Volvo"
]

In [14]:
Link_ERROR = 0
Read_announcements = 0
new_row = {}
all_data = []
for brand in car_brand:
    brand_link = f"https://www.otomoto.pl/osobowe/{brand}/?search%5Border%5D=created_at%3Adesc&page="
    for page_main in range(1,2): 
        try:
            html_data_link = requests.get(brand_link + str(page_main))
            soup_url = BeautifulSoup(html_data_link.content, 'html5lib')
        except Exception:
            print("Link_ERROR")
            continue
        links = []
        for offer in soup_url.find_all('a',attrs={"class":"offer-title__link"}):
            if "https://www.otomoto.pl" in offer['href']:
                links.append(offer['href']) 
        for page in links:
            print(page)
            html_data = requests.get(page)
            soup = BeautifulSoup(html_data.content, 'html5lib')
            dict_all = take_all(soup)
            dict_all = data_clean(dict_all)
            if dict_all:
                all_data.append(dict_all)
            Read_announcements += 1
            clear_output(wait=True)
            print("Read_announcements ", Read_announcements)
table_cars = pd.DataFrame.from_dict(all_data, orient='columns')
delete_duplicates = Read_announcements - len(table_cars.index) 
print("Delete duplicates: ", delete_duplicates)
print("Finish")

Read_announcements  192
Delete duplicates:  0
Finish


In [15]:
all_data

[{'ID': '6088515478',
  'price': 61900.0,
  'Oferta_od': 'Firmy',
  'Kategoria': 'Osobowe',
  'Marka_pojazdu': 'Abarth',
  'Model_pojazdu': '595',
  'Rok_produkcji': '2017',
  'Przebieg': '30631',
  'Pojemnosc_skokowa': '1368',
  'Rodzaj_paliwa': 'Benzyna',
  'Moc': '145',
  'Skrzynia_biegow': 'Manualna',
  'Naped': 'Na przednie koła',
  'Typ_nadwozia': 'Auta małe',
  'Liczba_drzwi': '3',
  'Liczba_miejsc': '4',
  'Kolor': 'Czerwony',
  'VAT_marza': 'Tak',
  'Mozliwosc_finansowania': 'Tak',
  'Leasing': 'Tak',
  'Bezwypadkowy': 'Tak',
  'Stan': 'Używane'},
 {'ID': '6088033302',
  'price': 71900.0,
  'Oferta_od': 'Firmy',
  'Kategoria': 'Osobowe',
  'Marka_pojazdu': 'Abarth',
  'Model_pojazdu': '595',
  'Rok_produkcji': '2018',
  'Przebieg': '25191',
  'Pojemnosc_skokowa': '1368',
  'Rodzaj_paliwa': 'Benzyna',
  'Moc': '145',
  'Skrzynia_biegow': 'Manualna',
  'Naped': 'Na przednie koła',
  'Typ_nadwozia': 'Auta małe',
  'Liczba_drzwi': '3',
  'Liczba_miejsc': '4',
  'Kolor': 'Szary',
 

In [16]:
# page_link = brand_link + str(page_main)
# print(page_link)

In [17]:
# table_cars = pd.DataFrame.from_dict(all_data, orient='columns')
# table_cars.head()

In [18]:
# #GIT
# Link_ERROR = 0
# Read_announcements = 0
# new_row = {}
# all_data = []
# for page_main in range(1,4):  
#     try:
#         html_data_link = requests.get(f"https://www.otomoto.pl/osobowe/?search%5Border%5D=created_at%3Adesc&page={page_main}")
#         soup_url = BeautifulSoup(html_data_link.content, 'html5lib')
#     except Exception:
#         print("Link_ERROR")
#         continue
#     links = []
#     for offer in soup_url.find_all('a',attrs={"class":"offer-title__link"}):
#         if "https://www.otomoto.pl" in offer['href']:
#             links.append(offer['href']) 
#     for page in links:
#         html_data = requests.get(page)
#         soup = BeautifulSoup(html_data.content, 'html5lib')
#         dict_all = take_all(soup)
#         dict_all = data_clean(dict_all)
#         all_data.append(dict_all)
#         Read_announcements += 1
#         clear_output(wait=True)
#         print("Read_announcements ", Read_announcements)
# table_cars = pd.DataFrame.from_dict(all_data, orient='columns')
# delete_duplicates = Read_announcements - len(table_cars.index) 
# print("Delete duplicates: ", delete_duplicates)
# print("Finish")

In [19]:
table_cars.head(50)

,ID,price,Oferta_od,Kategoria,Marka_pojazdu,Model_pojazdu,Rok_produkcji,Przebieg,Pojemnosc_skokowa,Rodzaj_paliwa,...,Emisja_CO2,Wersja,Spalanie_W_Cyklu_Mieszanym,Numer_rejestracyjny_pojazdu,Generacja,Spalanie_W_Miescie,Spalanie_Poza_Miastem,Okres_gwarancji_producenta,Gwarancja_dealerska_w_cenie,Miesieczna_rata
0,6088515478,61900.0,Firmy,Osobowe,Abarth,595,2017,30631,1368,Benzyna,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6088033302,71900.0,Firmy,Osobowe,Abarth,595,2018,25191,1368,Benzyna,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6088506602,67900.0,Firmy,Osobowe,Abarth,595,2019,18993,1368,Benzyna,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6085005657,93900.0,Firmy,Osobowe,Abarth,595,2021,1,1368,Benzyna,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6088473397,79900.0,Firmy,Osobowe,Abarth,595,2021,2,1368,Benzyna,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6088471928,34099.0,Firmy,Osobowe,Abarth,595,2017,42000,1368,Benzyna,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6088122709,106200.0,Firmy,Osobowe,Abarth,595,2021,1,1368,Benzyna,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,6088120797,93500.0,Firmy,Osobowe,Abarth,595,2021,1,1368,Benzyna,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,6088119494,97300.0,Firmy,Osobowe,Abarth,595,2021,1,1368,Benzyna,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,6088118229,89990.0,Firmy,Osobowe,Abarth,595,2021,300,1400,Benzyna,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
for col in table_cars.columns:
    print(col)

ID
price
Oferta_od
Kategoria
Marka_pojazdu
Model_pojazdu
Rok_produkcji
Przebieg
Pojemnosc_skokowa
Rodzaj_paliwa
Moc
Skrzynia_biegow
Naped
Typ_nadwozia
Liczba_drzwi
Liczba_miejsc
Kolor
VAT_marza
Mozliwosc_finansowania
Leasing
Bezwypadkowy
Stan
Kraj_pochodzenia
Faktura_VAT
Rodzaj_koloru
Uszkodzony
Zarejestrowany_w_Polsce
Serwisowany_w_ASO
Pierwsza_rejestracja
Pierwszy_wlasciciel
Emisja_CO2
Wersja
Spalanie_W_Cyklu_Mieszanym
Numer_rejestracyjny_pojazdu
Generacja
Spalanie_W_Miescie
Spalanie_Poza_Miastem
Okres_gwarancji_producenta
Gwarancja_dealerska_w_cenie
Miesieczna_rata


In [29]:
for col in insert_table:
    print(col)

ID
Oferta_od
Kategoria
Marka_pojazdu
Model_pojazdu
Wersja
Generacja
Rok_produkcji
Przebieg
Pojemnosc_skokowa
Rodzaj_paliwa
Moc
Skrzynia_biegow
Naped
Typ_nadwozia
Emisja_CO2
Liczba_drzwi
Liczba_miejsc
Kolor
Rodzaj_koloru
VAT_marza
Zarejestrowany_w_Polsce
Pierwszy_wlasciciel
Bezwypadkowy
Stan
Kraj_pochodzenia
Pierwsza_rejestracja
Serwisowany_w_ASO
Mozliwosc_finansowania
Faktura_VAT
Okres_gwarancji_producenta
Spalanie_W_Miescie
Numer_rejestracyjny_pojazdu
Leasing
Filtr_czastek_stalych
Spalanie_W_Cyklu_Mieszanym
Spalanie_Poza_Miastem
Uszkodzony
Gwarancja_dealerska_w_cenie
Miesieczna_rata


In [20]:
table_name = 'Otomoto'

In [21]:
table_cars.to_csv (table_name +'.csv', index = False, header=True)

In [22]:
psql = psycopg2.connect(host='192.168.10.163', port='5432', database='Otomoto', user='barto', password='biznes')

In [23]:
cur = psql.cursor()

In [24]:
cur.execute('DROP TABLE IF EXISTS '+ table_name)
createTableCommand = 'CREATE TABLE ' + table_name + '''(ID int NOT NULL,
    Oferta_od VARCHAR,
    Kategoria VARCHAR,
    Marka_pojazdu VARCHAR,
    Model_pojazdu VARCHAR,
    Wersja VARCHAR,
    Generacja VARCHAR,
    Rok_produkcji float,
    Przebieg float,
    Pojemnosc_skokowa float,
    Rodzaj_paliwa VARCHAR,
    Moc float,
    Skrzynia_biegow VARCHAR,
    Naped VARCHAR,
    Typ_nadwozia VARCHAR,
    Emisja_CO2 float,
    Liczba_drzwi float,
    Liczba_miejsc float,
    Kolor VARCHAR,
    Rodzaj_koloru VARCHAR,
    VAT_marza VARCHAR,
    Zarejestrowany_w_Polsce VARCHAR,
    Pierwszy_wlasciciel VARCHAR,
    Bezwypadkowy VARCHAR,
    Stan VARCHAR,
    Kraj_pochodzenia VARCHAR,
    Pierwsza_rejestracja VARCHAR,
    Serwisowany_w_ASO VARCHAR,
    Mozliwosc_finansowania VARCHAR,
    Faktura_VAT VARCHAR,
    Okres_gwarancji_producenta VARCHAR,
    Spalanie_W_Miescie float,
    Numer_rejestracyjny_pojazdu VARCHAR,
    Leasing VARCHAR,
    Filtr_czastek_stalych VARCHAR,
    Spalanie_W_Cyklu_Mieszanym float,
    Spalanie_Poza_Miastem float,
    Uszkodzony VARCHAR,
    Gwarancja_dealerska_w_cenie VARCHAR,
    Miesieczna_rata VARCHAR,
    PRIMARY KEY (ID)
    )'''
cur.execute(createTableCommand)
psql.commit()

In [25]:
print(table_cars.columns)

Index(['ID', 'price', 'Oferta_od', 'Kategoria', 'Marka_pojazdu',
       'Model_pojazdu', 'Rok_produkcji', 'Przebieg', 'Pojemnosc_skokowa',
       'Rodzaj_paliwa', 'Moc', 'Skrzynia_biegow', 'Naped', 'Typ_nadwozia',
       'Liczba_drzwi', 'Liczba_miejsc', 'Kolor', 'VAT_marza',
       'Mozliwosc_finansowania', 'Leasing', 'Bezwypadkowy', 'Stan',
       'Kraj_pochodzenia', 'Faktura_VAT', 'Rodzaj_koloru', 'Uszkodzony',
       'Zarejestrowany_w_Polsce', 'Serwisowany_w_ASO', 'Pierwsza_rejestracja',
       'Pierwszy_wlasciciel', 'Emisja_CO2', 'Wersja',
       'Spalanie_W_Cyklu_Mieszanym', 'Numer_rejestracyjny_pojazdu',
       'Generacja', 'Spalanie_W_Miescie', 'Spalanie_Poza_Miastem',
       'Okres_gwarancji_producenta', 'Gwarancja_dealerska_w_cenie',
       'Miesieczna_rata'],
      dtype='object')


In [26]:
insert_table = ['ID',
    'Oferta_od',
    'Kategoria',
    'Marka_pojazdu',
    'Model_pojazdu',
    'Wersja',
    'Generacja',
    'Rok_produkcji',
    'Przebieg',
    'Pojemnosc_skokowa',
    'Rodzaj_paliwa',
    'Moc',
    'Skrzynia_biegow',
    'Naped',
    'Typ_nadwozia',
    'Emisja_CO2',
    'Liczba_drzwi',
    'Liczba_miejsc',
    'Kolor',
    'Rodzaj_koloru',
    'VAT_marza',
    'Zarejestrowany_w_Polsce',
    'Pierwszy_wlasciciel',
    'Bezwypadkowy',
    'Stan',
    'Kraj_pochodzenia',
    'Pierwsza_rejestracja',
    'Serwisowany_w_ASO',
    'Mozliwosc_finansowania',
    'Faktura_VAT',
    'Okres_gwarancji_producenta',
    'Spalanie_W_Miescie',
    'Numer_rejestracyjny_pojazdu',
    'Leasing',
    'Filtr_czastek_stalych',
    'Spalanie_W_Cyklu_Mieszanym',
    'Spalanie_Poza_Miastem',
    'Uszkodzony',
    'Gwarancja_dealerska_w_cenie',
    'Miesieczna_rata']


a = 0
for row in table_cars.itertuples():
    full=""
    full_fit="INSERT INTO " + table_name + "("
    for (column, insert) in zip(row, insert_table):
        if str(column) =='nan':
            #print("Fuck")
            continue
        column = str(column)
        column = column.replace("'", "''")
        full +="'"+column+"'"+", "
        full_fit += insert + ', '  

    full=full[:-2]
    full_fit=full_fit[:-2]
    full_fit=full_fit+')'
    Value_data='VALUES ('+full+');'
    insertDataCommand = full_fit+"\r\n"+Value_data
    print(insertDataCommand)
    cur.execute(insertDataCommand)
psql.commit()
print("Finish")

INSERT INTO Otomoto(ID, Oferta_od, Kategoria, Marka_pojazdu, Model_pojazdu, Wersja, Generacja, Rok_produkcji, Przebieg, Pojemnosc_skokowa, Rodzaj_paliwa, Moc, Skrzynia_biegow, Naped, Typ_nadwozia, Emisja_CO2, Liczba_drzwi, Liczba_miejsc, Kolor, Rodzaj_koloru, VAT_marza, Zarejestrowany_w_Polsce, Pierwszy_wlasciciel)
VALUES ('0', '6088515478', '61900.0', 'Firmy', 'Osobowe', 'Abarth', '595', '2017', '30631', '1368', 'Benzyna', '145', 'Manualna', 'Na przednie koła', 'Auta małe', '3', '4', 'Czerwony', 'Tak', 'Tak', 'Tak', 'Tak', 'Używane');


InvalidTextRepresentation: invalid input syntax for type double precision: "Czerwony"
LINE 2: ...alna', 'Na przednie koła', 'Auta małe', '3', '4', 'Czerwony'...
                                                             ^
